## This notebook explores the adiabatic method

In [ ]:
using Bloqade
using Random
Random.seed!(42)
using PythonCall
plt = pyimport("matplotlib.pyplot");

In [ ]:
Random.seed!(2)
a=4.5
atoms = generate_sites(SquareLattice(), 4,4; scale = a) |> random_dropout(0.2)

In [ ]:
Rb=sqrt(3*sqrt(5))*a
blockade_radius = Rb

In [ ]:
Bloqade.plot(atoms, blockade_radius = Rb)

In [ ]:
#Total evolution time
T_max = 3

#Fix Δ according to the desired Rydberg radius
C6 = 2π * 862690;
Ω_max = 2π*2.5
Δ_max = C6 / Rb^6
a=.8
#Build time traces for adiabatic algorithm
Ω = piecewise_linear(clocks = [0.0, 0.25, 2.75, T_max], values = [0.0, Ω_max, Ω_max, 0])
Δ_start = -Δ_max
Δ_end = Δ_max
# Δ = piecewise_linear(clocks = [0.0, 0.75, 2.25, T_max], values = [Δ_start, Δ_start, Δ_end, Δ_end])
w1 = constant(duration=0.25, value=Δ_start)
# w2 = Waveform( t ->  (a * (t)^3) - (4.5*a*(t)^2) + (6.75 * a * (t)) + Δ_max - (5.327*a), duration = 2.5)
w2 = Waveform(t->(((1.36*sin((1.05*t)-(3.14/2)))^3) + (2.162/3.14*(0.4))), duration = 2.5)
w3 = constant(duration=0.25, value=Δ_end)
Δ = append(append(w1, w2), w3)
#Quick plot to check
fig, (ax1, ax2) = plt.subplots(ncols = 2, figsize = (12, 4))
Bloqade.plot!(ax1, Ω)
ax1.set_ylabel("Ω/2π (MHz)")
Bloqade.plot!(ax2, Δ)
ax2.set_ylabel("Δ/2π (MHz)")
fig

In [ ]:
h_kings = rydberg_h(atoms; Ω = Ω, Δ = Δ)
prob = SchrodingerProblem(zero_state(nqubits(h_kings)), T_max, h_kings);

In [ ]:
emulate!(prob)

In [ ]:
bitstring_hist(prob.reg; nlargest = 10)

In [ ]:
best_bit_strings = most_probable(prob.reg, 3)

Bloqade.plot(atoms, blockade_radius = Rb; colors = [iszero(b) ? "white" : "red" for b in best_bit_strings[1]])